In [1]:
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings
warnings.filterwarnings("ignore")

In [2]:
TRAIN_PATH = 'C:/Users/Anatoly/Desktop/Python_and_DS/file_train.csv'
TEST_PATH = 'C:/Users/Anatoly/Desktop/Python_and_DS/file_test.csv'

In [3]:
df_train = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TRAIN_PATH)

# Отбор признаков

In [4]:
TARGET_NAME = 'Credit Default'
BASE_FEATURE_NAMES = df_train.columns.drop(TARGET_NAME).tolist()

In [5]:
BASE_FEATURE_NAMES

['Unnamed: 0',
 'Home Ownership',
 'Annual Income',
 'Years in current job',
 'Tax Liens',
 'Number of Open Accounts',
 'Years of Credit History',
 'Maximum Open Credit',
 'Number of Credit Problems',
 'Months since last delinquent',
 'Bankruptcies',
 'Purpose',
 'Term',
 'Current Loan Amount',
 'Current Credit Balance',
 'Monthly Debt',
 'Credit Score',
 'Number of years in current job',
 'Home Ownership_Have Mortgage',
 'Home Ownership_Home Mortgage',
 'Home Ownership_Own Home',
 'Home Ownership_Rent',
 'ID',
 'Classify_purpose']

In [6]:
NUM_FEATURE_NAMES = ['Number of Open Accounts', 'Years of Credit History', 'Maximum Open Credit', 
                     'Number of Credit Problems', 'Months since last delinquent', 'Bankruptcies', 'Current Loan Amount', 
                     'Current Credit Balance', 'Monthly Debt', 'Credit Score', 'Number of years in current job', 'ID']

CAT_FEATURE_NAMES = [ 'Term', 'Classify_purpose',  'Home Ownership_Have Mortgage', 'Home Ownership_Home Mortgage',
                     'Home Ownership_Own Home', 'Home Ownership_Rent']

In [7]:
SELECTED_FEATURE_NAMES = NUM_FEATURE_NAMES + CAT_FEATURE_NAMES

In [8]:
#!pip install xgboost
#!pip install lightgbm
#!pip install catboost
#!pip install imblearn

In [27]:
import pandas as pd
import numpy as np
import pickle
import random

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score, learning_curve
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score as f1, precision_score, recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb, lightgbm as lgbm, catboost as catb

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [10]:
import warnings
warnings.simplefilter('ignore')

In [11]:
def norm_df(df_train, df_test):
    scaler = StandardScaler()
    df_train[NUM_FEATURE_NAMES] = scaler.fit_transform(df_train[NUM_FEATURE_NAMES])
    df_test[NUM_FEATURE_NAMES] = scaler.transform(df_test[NUM_FEATURE_NAMES])
    return df_train, df_test

In [12]:
df_train, df_test = norm_df(df_train, df_test)

In [13]:
df_train[SELECTED_FEATURE_NAMES].head()

,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Number of years in current job,ID,Term,Classify_purpose,Home Ownership_Have Mortgage,Home Ownership_Home Mortgage,Home Ownership_Own Home,Home Ownership_Rent
0,0.061171,1.167923,0.684156,2.504893,-0.698538,2.842517,2.760520,-1.017387,-0.956697,0.608640,-1.240797,-1.731820,0,-1.0,0,0,1,0
1,1.008578,-0.428100,2.443371,-0.399219,-0.698538,-0.351801,-0.363620,0.621818,0.103274,0.567701,1.364411,-1.731358,1,-1.0,0,0,1,0
2,0.061171,2.430231,2.445869,-0.399219,-0.698538,-0.351801,2.760520,0.213495,-0.375279,0.584759,0.148647,-1.730896,0,-1.0,0,1,0,0
3,-0.649385,0.616569,-1.226897,2.504893,-0.698538,2.842517,-0.368118,-0.788808,-0.609691,0.421002,-0.198714,-1.730434,0,-1.0,0,0,1,0
4,0.534875,-0.674758,-0.380819,2.504893,-0.698538,-0.351801,-0.367978,-0.800815,-1.031085,0.506292,0.148647,-1.729972,0,-1.0,0,0,0,1


In [16]:
X = df_train[SELECTED_FEATURE_NAMES]
y = df_train[TARGET_NAME]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, shuffle=True, test_size=0.3, random_state=42)

In [19]:
df_for_balancing = pd.concat([X_train, y_train], axis=1)
df_for_balancing[TARGET_NAME].value_counts()

0    3795
1    1455
Name: Credit Default, dtype: int64

In [20]:
def balance_df_by_target(df, target_name):

    target_counts = df[target_name].value_counts()

    major_class_name = target_counts.argmax()
    minor_class_name = target_counts.argmin()

    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1

    for i in range(disbalance_coeff):
        sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
        df = df.append(sample, ignore_index=True)

    return df.sample(frac=1)

# Балансировка классов

In [21]:
df_balanced = balance_df_by_target(df_for_balancing, TARGET_NAME)
df_balanced[TARGET_NAME].value_counts()

0    3795
1    2910
Name: Credit Default, dtype: int64

In [22]:
X_train = df_balanced.drop(columns=TARGET_NAME)
y_train = df_balanced[TARGET_NAME]

# Создание моделей

In [36]:
def create_model(df_train, df_valid, y_train, y_valid, model, features):
    model.fit(df_train.loc[:, features], y_train.values.ravel())
    pred_valid = model.predict(df_valid.loc[:, features])
    f1_ = f1(y_valid, pred_valid)
    print(f'f1: {f1_}')
    return (pred_valid)

In [37]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
pred_knn_train = create_model(X_train, X_train, y_train, y_train.values.ravel(), knn, SELECTED_FEATURE_NAMES)
pred_knn_valid = create_model(X_train, X_valid, y_train, y_valid.values.ravel(), knn, SELECTED_FEATURE_NAMES)

f1: 0.8585337070364361
f1: 0.40756558877364246


In [38]:
rf = RandomForestClassifier(n_estimators=300, max_depth=8, random_state=42, max_features=8, min_samples_leaf=2)
pred_rf_train = create_model(X_train, X_train, y_train, y_train, rf, SELECTED_FEATURE_NAMES)
pred_rf_valid = create_model(X_train, X_valid, y_train, y_valid, rf, SELECTED_FEATURE_NAMES)

f1: 0.6683046683046682
f1: 0.42712392829306306


In [39]:
from sklearn.svm import SVC
svc_model = SVC(gamma='scale', C=0.1)
pred_svc_train = create_model(X_train, X_train, y_train, y_train.values.ravel(), svc_model, SELECTED_FEATURE_NAMES)
pred_svc_valid = create_model(X_train, X_valid, y_train, y_valid.values.ravel(), svc_model, SELECTED_FEATURE_NAMES)

f1: 0.49543233482048005
f1: 0.3766122098022356


In [40]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs')
pred_lr_train = create_model(X_train, X_train, y_train, y_train.values.ravel(), lr, SELECTED_FEATURE_NAMES)
pred_lr_valid = create_model(X_train, X_valid, y_train, y_valid.values.ravel(), lr, SELECTED_FEATURE_NAMES)

f1: 0.5373480023161552
f1: 0.4327573253193088


Лучше всех отработал KNN, RandomForestClassifier, LogisticRegression. Получен бейзлайн, но пока ниже целевого значения 0.5. Как улучшить: поработать еще с признаками (вижу тут потенциал), попробовать кросс-валидацию, xgboost. 

Надо еще раз проанализировать "Annual income", здесь есть потенциал. Также нужно проверить, можно ли объединить Home Ownership в один через dummy переменные. Также в предыдущем уроке мы убедились, что некоторые числовые переменные можно прологарифмировать и представить через преобразование boxcox.

Еще можно попробовать балансировку классов через Imblearn. 

- Для чего и в каких случаях полезны различные варианты усреднения для метрик качества классификации: micro, macro, weighted?
- В чём разница между моделями xgboost, lightgbm и catboost или какие их основные особенности?

По второму вопросу: 

https://towardsdatascience.com/catboost-vs-light-gbm-vs-xgboost-5f93620723db

https://github.com/Pyligent/lightGBM_vs_XGBoost

В первой статье есть даже табличное сравнение по входным параметрам в модель. Если коротко, то lightgbm использует новую технику односторонней выборки на основе градиента (GOSS), xgboost - алгоритм на основе предварительной сортировки для поиска наилучшего разбиения. Catboost - модель машинного обучения на основе градиентного бустинга. Эта модель точнее и может работать с категориальными переменными напрямую, преобразовывать не надо. 
https://habr.com/ru/company/yandex/blog/458790/


"Категориальные фичи мы процессим немножечко по-другому. Категориальные фичи мы в самом начале хэшируем, а потом перенумеровываем хэши от нуля до количества уникальных значений категориальной фичи, чтобы быстро считать комбинации категориальных фич." - из доклада на Хабре. 

По первому вопросу: 

Оригинал: https://github.com/scikit-learn/scikit-learn/blob/7b136e9/sklearn/metrics/classification.py#L620

        ``'Микро'``:
            Вычисление метрик по всему датасету, подсчет полного кол-ва TP, FN, FP. 
            Исходя из определения, используем повсеместно. 
        ``'Макро'``:
            Вычисление метрики для каждого признака и поиск его средней невзвешенной величины. 
            В этом случае не учитывается дисбаланс признака. 
        ``'Взвешенный'``:
            Вычисление метрики для каждого признака и поиск среднего взвешенного
            (кол-во истинных значений для каждого признака)

"Взвешенный" используется В случае неравных классов нужно подбирать баланс классов для обучения и метрику, которая будет корректно отражать качество классификации 

Чтобы точно донести мысль как считается f1 при микро и макро подходе, обратимся к статье: http://rushdishams.blogspot.com/2011/08/micro-and-macro-average-of-precision.html

Если коротко, то при микро мы считаем метрики на уровне всех компонент (TP1, TP2, FN1, FN2....), а при макро - сразу получаем данные precision и recall, после усредняем. 

Макро подход позволяет оценивать модель с точки зрения его устойчивости. Микро - разбираться на уровне признаков. 